In [4]:
from __future__ import print_function

In [2]:
import torch

## 1.基础

### 1.1简单开始

#### 生成一个空矩阵

In [5]:
x=torch.empty(5,3)
print(x)

tensor([[4.5001e-39, 4.7755e-39, 5.0510e-39],
        [4.2246e-39, 1.0286e-38, 1.0653e-38],
        [1.0194e-38, 8.4490e-39, 1.0469e-38],
        [9.3674e-39, 9.9184e-39, 8.7245e-39],
        [9.2755e-39, 8.9082e-39, 9.9184e-39]])


#### 创建一个随机矩阵

In [7]:
x=torch.rand(5,3)
print(x)

tensor([[0.5522, 0.6123, 0.9351],
        [0.4079, 0.0797, 0.0976],
        [0.3334, 0.6595, 0.3532],
        [0.1986, 0.1479, 0.3021],
        [0.8412, 0.9560, 0.5985]])


#### 创建0矩阵

In [8]:
x=torch.zeros(5,3,dtype=torch.long)
print(x)

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])


#### 创建tensor并初始化

In [9]:
x=torch.tensor([5.5,3])
print(x)

tensor([5.5000, 3.0000])


#### 重定义现有张量

In [11]:
x=x.new_ones(5,3,dtype=torch.double)
print(x)

x=torch.randn_like(x,dtype=torch.float)
print(x)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)
tensor([[ 0.8728, -0.7331, -0.6845],
        [-0.2447,  0.5041, -0.2797],
        [-0.1924,  0.7538, -0.3251],
        [-0.0250, -0.1499, -0.1418],
        [-0.0300,  0.1481,  0.5381]])


#### 获取size

In [12]:
print(x.size())

torch.Size([5, 3])


注意：torch.size返回值是turple类型

#### 操作：加法

In [20]:
y=torch.rand(5,3)

print(x+y)

print(torch.add(x,y))

result=torch.empty(5,3)
torch.add(x,y,out=result)

y.add_(x)

tensor([[ 1.7479, -0.5857, -0.5298],
        [-0.0883,  1.2303,  0.6369],
        [ 0.0684,  0.9854,  0.5601],
        [ 0.8246,  0.3008,  0.6069],
        [ 0.9674,  0.5192,  0.9856]])
tensor([[ 1.7479, -0.5857, -0.5298],
        [-0.0883,  1.2303,  0.6369],
        [ 0.0684,  0.9854,  0.5601],
        [ 0.8246,  0.3008,  0.6069],
        [ 0.9674,  0.5192,  0.9856]])


tensor([[ 1.7479, -0.5857, -0.5298],
        [-0.0883,  1.2303,  0.6369],
        [ 0.0684,  0.9854,  0.5601],
        [ 0.8246,  0.3008,  0.6069],
        [ 0.9674,  0.5192,  0.9856]])

#### 索引：与numoy类似操作

In [21]:
print(x[:,1])

tensor([-0.7331,  0.5041,  0.7538, -0.1499,  0.1481])


#### 改变数组结构：reshape->view

In [23]:
x=torch.randn(4,4)
y=x.view(16)
z=x.view(-1,8)
print(x.size(),y.size(),z.size())

torch.Size([4, 4]) torch.Size([16]) torch.Size([2, 8])


### 1.2 torch与numpy转换

简单：torch tensor <-> numpy array

In [24]:
a=torch.ones(5,5)
b=a.numpy()
print(b)

[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]


In [28]:
import numpy as np
a=np.ones((5,5))
b=torch.from_numpy(a)
print(b)

tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]], dtype=torch.float64)


### 1.3 CUDA张量

使用.to方法可以将Tensor移动到任何设备中

In [36]:
if torch.cuda.is_available():
    device=torch.device("cuda") #a CUDA设备对象
    y=torch.ones_like(x,device=device) #从GPU创建张量
    x=x.to(device) #直接将张量移动到cuda中
    z=x+y
    print(z)
    
else:
    print("cuda error")

cuda error


## 2.Autograd: 自动求导机制

PyTorch 中所有神经网络的核心是 autograd包。autograd包为张量上的所有操作提供了自动求导。 它是一个在运行时定义的框架，这意味着反向传播是根据你的代码来确定如何运行，并且每次迭代可以是不同的。

* requires_grad=True/False
* grad
* grad_fn
* is_leaf


* 张量创建时，通过设置 requires_grad 标识为Ture来告诉Pytorch需要对该张量进行自动求导，PyTorch会记录该a张量的每一步操作历史并自动计算
* PyTorch会自动追踪和记录对与张量的所有操作，当计算完成后调用.backward()方法自动计算梯度并且将计算结果保存到grad属性中。
* 在张量进行操作后，grad_fn已经被赋予了一个新的函数，这个函数引用了一个创建了这个Tensor类的Function对象。 Tensor和Function互相连接生成了一个非循环图，它记录并且编码了完整的计算历史。每个张量都有一个.grad_fn属性，如果这个张量是用户手动创建的那么这个张量的grad_fn是None。
* 一个变量是创建变量还是结果变量是通过.is_leaf来获取
* grad_fn：记录并且编码了完整的计算历史

### 2.1 张量（Tensor）

torch.Tensor是这个包的核心类。如果设置 .requires_grad 为 True，那么将会追踪所有对于该张量的操作。 当完成计算后通过调用 .backward()，自动计算所有的梯度， 这个张量的所有梯度将会自动积累到 .grad 属性。**require_grad:这个标志表明这个tensor的操作是否会被pytorch的自动微分系统（Autograd）记录其操作过程，以便后续自动求导。**

要阻止张量跟踪历史记录，可以调用.detach()方法将其与计算历史记录分离，并禁止跟踪它将来的计算记录。

为了防止跟踪历史记录（和使用内存），可以将代码块包装在with torch.no_grad()：中。 在评估模型时特别有用，因为模型可能具有requires_grad = True的可训练参数，但是我们不需要梯度计算。

在自动梯度计算中还有另外一个重要的类Function.
Tensor 和 Function互相连接并生成一个非循环图，它表示和存储了完整的计算历史。 每个张量都有一个.grad_fn属性，这个属性引用了一个创建了Tensor的Function（除非这个张量是用户手动创建的，即，这个张量的 grad_fn 是 None）

如果需要计算导数，你可以在Tensor上调用.backward()。 如果Tensor是一个标量（即它包含一个元素数据）则不需要为backward()指定任何参数， 但是如果它有更多的元素，你需要指定一个gradient 参数来匹配张量的形状

#### 创建一个张量并设置追踪计算历史

In [64]:
x=torch.ones(2,2,requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [72]:
y=x+2
print(y)
print(y.grad_fn)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [73]:
z=y*y*3
out=z.mean()
print(z,out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


.requires_grad_( ... ) 可以改变现有张量的 requires_grad属性。 如果没有指定的话，默认输入的flag是 False。

In [74]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
True


### 2.2 梯度

In [75]:
print(out)
out.backward()

tensor(27., grad_fn=<MeanBackward0>)


In [71]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


正向求解过程：
$$x -> y -> z -> out$$
$$y=x+2$$
$$z=y*y*3$$
$$out=mean(z)$$

反向传播过程：
$$out=\frac{1}{4}{\sum _{i}{z_i}}$$
$$z_i=3(x_i+2)^2$$
$$z_i|_{x_i=1}=27$$
$$\frac {\partial out}{\partial x_i}{=\frac {3}{2}{(x_i+2)}}=4.5$$

雅可比矩阵(Jacobian matrix)：$$J=\left[ \begin{matrix} {\frac {\partial y_1}{\partial x_1}}{ ... }{\frac {\partial y_1}{\partial x_n}}\\{\frac {\partial y_m}{\partial x_1}}{ ... }{\frac {\partial f_2}{\partial y}}\end{matrix} \right]$$

一般来说，torch.autograd就是用来计算vector-Jacobian product的工具
给定任意向量$c={({\frac {\partial l}{\partial y_1}}{\frac {\partial l}{\partial m}})}^T$,计算$v_T*J$
如果v恰好是**标量函数**$l=g(y)$的梯度，那么根据链式法则，vector-Jacobian product 是 $l$ 关于 $\vec{x}$ 的梯度：
$$J^{T}\cdot v = \begin{pmatrix} \frac{\partial y_{1}}{\partial x_{1}} &amp; \cdots &amp; \frac{\partial y_{m}}{\partial x_{1}} \\ \vdots &amp; \ddots &amp; \vdots \\ \frac{\partial y_{1}}{\partial x_{n}} &amp; \cdots &amp; \frac{\partial y_{m}}{\partial x_{n}} \end{pmatrix} \begin{pmatrix} \frac{\partial l}{\partial y_{1}}\\ \vdots \\ \frac{\partial l}{\partial y_{m}} \end{pmatrix} = \begin{pmatrix} \frac{\partial l}{\partial x_{1}}\\ \vdots \\ \frac{\partial l}{\partial x_{n}} \end{pmatrix}$$
vector-Jacobian product 这种特性使得将外部梯度返回到具有非标量输出的模型变得非常方便。

#### vector-Jacobian product的例子

In [9]:
x=torch.randn(3,requires_grad=True)
print(x)
y=x*2
while y.data.norm()<1000:
    print(y.data.norm())#y张量L2范数，先对y中每一项取平方，之后累加，最后取根号
    #print(y)
    y=y*2
print(y)

tensor([-1.0432,  0.7504,  1.8048], requires_grad=True)
tensor(4.4311)
tensor(8.8621)
tensor(17.7243)
tensor(35.4485)
tensor(70.8971)
tensor(141.7942)
tensor(283.5883)
tensor(567.1767)
tensor([-534.1100,  384.2012,  924.0528], grad_fn=<MulBackward0>)



在这个情形中，y不再是个标量。torch.autograd无法直接计算出完整的雅可比行列，但是如果我们只想要vector-Jacobian product，只需将向量作为参数传入backward：

In [12]:
gradients=torch.tensor([0.1,1.0,0.0001],dtype=torch.float)
print(gradients)
y.backward(gradients)
print(x.grad)

tensor([1.0000e-01, 1.0000e+00, 1.0000e-04])
tensor([5.1200e+01, 5.1200e+02, 5.1200e-02])


如果.requires_grad=True但是你又不希望进行autograd的计算， 那么可以将变量包裹在 with torch.no_grad()中:

In [14]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
	print((x ** 2).requires_grad)

True
True
False


## 3.神经网络包和优化器optm

torch.nn是专门为神经网络设计的模块化接口。nn构建于 Autograd之上，可用来定义和运行神经网络。 这里我们主要介绍几个一些常用的类.
nn.functional，这个包中包含了神经网络中使用的一些常用函数，这些函数的特点是，不具有可学习的参数(如ReLU，pool，DropOut等)，这些函数可以放在构造函数中，也可以不放，但是这里建议不放。

In [20]:
import torch.nn as nn
import torch.nn.functional as F

In [16]:
torch.__version__

'1.7.1+cpu'

### 3.1定义一个网络

PyTorch中已经为我们准备好了现成的网络模型，只要继承nn.Module，并实现它的forward方法，PyTorch会根据autograd，自动实现backward函数，在forward函数中可使用任何tensor支持的函数，还可以使用if、for循环、print、log等Python语法，写法和标准的Python写法一致。

In [31]:
class Net(nn.Module):
    def __init__(self):
        
        #两种继承构造函数初始化方法
        #super(Net,self).__init__()
        nn.Module.__init__(self)
        
        self.conv1=nn.Conv2d(1,6,3)#卷积层：1通道，6输出通道，3卷积核3*3
        self.fc1=nn.Linear(1350,10)#线形层：
        
    def forward(self,x):
        print(x.size())
        
        x=self.conv1(x)
        x=F.relu(x)
        print(x.size())
        
        x=F.max_pool2d(x,(2,2))
        x=F.relu(x)
        print(x.size())
        
        x=x.view(x.size()[0],-1)
        print(x.size())
        
        x=self.fc1(x)
        
        return x

In [32]:
net=Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=1350, out_features=10, bias=True)
)


网络的参数通过net.parameters()返回

In [33]:
for parameters in net.parameters():
    print(parameters)

Parameter containing:
tensor([[[[ 0.2019,  0.1373,  0.2374],
          [ 0.2079, -0.2955,  0.2644],
          [ 0.2720,  0.0808,  0.0344]]],


        [[[ 0.0710,  0.0482, -0.0419],
          [ 0.2607, -0.3042,  0.1330],
          [ 0.0423, -0.0514,  0.1212]]],


        [[[-0.2516,  0.0331, -0.3285],
          [ 0.2308,  0.2670, -0.3221],
          [ 0.1206,  0.1483, -0.3107]]],


        [[[ 0.2866,  0.1840, -0.1916],
          [-0.2969, -0.1923, -0.2749],
          [-0.2781,  0.2501, -0.2626]]],


        [[[-0.0551, -0.1942, -0.1881],
          [ 0.3202, -0.0265,  0.2750],
          [ 0.2234, -0.0681, -0.2320]]],


        [[[-0.0057,  0.1044, -0.0820],
          [-0.0537, -0.0618,  0.0142],
          [ 0.2844, -0.3055, -0.0087]]]], requires_grad=True)
Parameter containing:
tensor([-0.1178, -0.0080,  0.1140,  0.1008,  0.1060, -0.0734],
       requires_grad=True)
Parameter containing:
tensor([[-0.0196,  0.0176,  0.0251,  ..., -0.0145,  0.0095,  0.0102],
        [-0.0011,  0.0175, -0

net.named_parameters可同时返回可学习的参数及名称。

In [34]:
for name,parameters in net.named_parameters():
    print(name,':',parameters.size())

conv1.weight : torch.Size([6, 1, 3, 3])
conv1.bias : torch.Size([6])
fc1.weight : torch.Size([10, 1350])
fc1.bias : torch.Size([10])


forward函数的输入和输出都是Tensor

In [35]:
input=torch.randn(1,1,32,32)
out=net(input)
out.size()

torch.Size([1, 1, 32, 32])
torch.Size([1, 6, 30, 30])
torch.Size([1, 6, 15, 15])
torch.Size([1, 1350])


torch.Size([1, 10])


在反向传播前，先要将所有参数的梯度清零

In [41]:
net.zero_grad()
out.backward(torch.ones(1,10))

### 3.2 损失函数

在nn中PyTorch还预制了常用的损失函数，下面我们用MSELoss用来计算均方误差

In [43]:
y = torch.arange(0,10).view(1,10).float()
criterion = nn.MSELoss()
loss = criterion(out, y)
#loss是个scalar，我们可以直接用item获取到他的python类型的数值
print(loss.item())

29.479848861694336


### 3.3优化器

在反向传播计算完所有参数的梯度后，还需要使用优化方法来更新网络的权重和参数，例如随机梯度下降法(SGD)的更新策略如下：

$$weight = weight - learning_rate * gradient$$

在torch.optim中实现大多数的优化方法，例如RMSProp、Adam、SGD等

In [44]:
import torch.optim

In [45]:
out = net(input) # 这里调用的时候会打印出我们在forword函数中打印的x的大小
criterion = nn.MSELoss()
loss = criterion(out, y)
#新建一个优化器，SGD只需要要调整的参数和学习率
optimizer = torch.optim.SGD(net.parameters(), lr = 0.01)
# 先梯度清零(与net.zero_grad()效果一样)
optimizer.zero_grad() 
loss.backward()

#更新参数
optimizer.step()

torch.Size([1, 1, 32, 32])
torch.Size([1, 6, 30, 30])
torch.Size([1, 6, 15, 15])
torch.Size([1, 1350])


## 4.数据的加载和预处理

PyTorch通过torch.utils.data对一般常用的数据加载进行了封装，可以很容易地实现多线程数据预读和批量加载。 并且torchvision已经预先实现了常用图像数据集，包括前面使用过的CIFAR-10，ImageNet、COCO、MNIST、LSUN等数据集，可通过torchvision.datasets方便的调用

### 4.1Dataset

Dataset是一个抽象类，为了能够方便的读取，需要将要使用的数据包装为Dataset类。 自定义的Dataset需要继承它并且实现两个成员方法：

1.__getitem__() 该方法定义用索引(0 到 len(self))获取一条数据或一个样本

2.__len__() 该方法返回数据集的总长度

In [46]:
from torch.utils.data import Dataset

In [47]:
import pandas as pd 

In [51]:
class BulldozerDataset(Dataset):
    def __init__(self):
        self.df=pd.read_csv(csv_file)
    def __len__(self):
        return len(self.df)
    def __getitem__(self,idx):
        return self.df.iloc[idx].SalePrice

In [52]:
ds_demo=BulldozerDataset('median_benchmark.csv')

TypeError: __init__() takes 1 positional argument but 2 were given

### 4.2Dataloader

DataLoader为我们提供了对Dataset的读取操作，常用参数有：batch_size(每个batch的大小)、 shuffle(是否进行shuffle操作)、 num_workers(加载数据的时候使用几个子进程)。下面做一个简单的操作

In [53]:
dl = torch.utils.data.DataLoader(ds_demo, batch_size=10, shuffle=True, num_workers=0)

NameError: name 'ds_demo' is not defined

In [54]:
idata=iter(dl)
print(next(idata))

NameError: name 'dl' is not defined

### 4.3torchvision

torchvision 是PyTorch中专门用来处理图像的库，PyTorch官网的安装教程中最后的pip install torchvision 就是安装这个包。

torchvision.datasets 可以理解为PyTorch团队自定义的dataset，这些dataset帮我们提前处理好了很多的图片数据集，我们拿来就可以直接使用：

* MNIST
* COCO
* Captions
* Detection
* LSUN
* ImageFolder
* Imagenet-12
* CIFAR
* STL10
* SVHN

In [56]:
import torchvision.datasets as datasets

ModuleNotFoundError: No module named 'torchvision'

In [55]:
trainset = datasets.MNIST(root='./data', # 表示 MNIST 数据的加载的目录
                                      train=True,  # 表示是否加载数据库的训练集，false的时候加载测试集
                                      download=True, # 表示是否自动下载 MNIST 数据集
                                      transform=None) # 表示是否需要对数据进行预处理，none为不进行预处理

NameError: name 'datasets' is not defined

torchvision.transforms
transforms 模块提供了一般的图像转换操作类，用作数据处理和数据增强

torchvision不仅提供了常用图片数据集，还提供了训练好的模型，可以加载之后，直接使用，或者在进行迁移学习 torchvision.models模块的 子模块中包含以下模型结构。

* AlexNet
* VGG
* ResNet
* SqueezeNet
* DenseNet